# The tests of IvdSlam, Loop-closure, and Segmentation

## 0. visualize images

In [2]:
FileList = glob.glob('F:/pfe_2/realsense/frames/rgb/*.npy')
i = 0
k = 0
bar = progressbar.ProgressBar(maxval=len(FileList), \
            widgets=[progressbar.Bar('=', '[', ']'), ' ', progressbar.Percentage()])
bar.start()
for imagef in FileList:
    image = np.load(imagef)
    cv2.imwrite('rgb/'+str(i).zfill(3)+'.jpg', image)
    bar.update(k+1)
    k += 1
    i += 1
bar.finish()

[========================================================================] 100%


![title](img/imgs.PNG)

## 1. Loop closure detection test
use the bag-of-words algorithm to detect (some functions are from opencv open source)

In [1]:
from LoopClosure import LoopClosure
import glob
import numpy as np
import cv2
import progressbar

### 1.1 init by training

In [15]:
# init class
loopclosure = LoopClosure()

FileList = glob.glob('F:/pfe_2/realsense/frames/rgb/*.npy')
# train the bag of words by the images of the file list
loopclosure.set_vocabulary_by_train(FileList)
# save the vocabulary
loopclosure.save_vocabulary('voca.npy')

--- start training ---
--- train finished ---


### 1.2 init by load trained BOW

In [2]:
loopclosure = LoopClosure()
loopclosure.set_vocabulary('voca.npy')

### 1.3 test

In [3]:
# to reset the image signatures
loopclosure.image_signatures = []
for i in range(150):
    image = np.load('F:/pfe_2/realsense/frames/rgb/'+str(i).zfill(3)+'.npy')
    inp = loopclosure.get_signature(image)
    
    d, n = loopclosure.loop_closure_detection(inp, i, 0.07, 20)
    #print(i,'if detected loop:', d, ',   if true with image num: ',n )
    if d :
        print(i,'if detected loop:', d, ',   if true with image num: ',n )

67 if detected loop: True ,   if true with image num:  46
69 if detected loop: True ,   if true with image num:  48
70 if detected loop: True ,   if true with image num:  46
71 if detected loop: True ,   if true with image num:  47
72 if detected loop: True ,   if true with image num:  49
73 if detected loop: True ,   if true with image num:  51
140 if detected loop: True ,   if true with image num:  35
141 if detected loop: True ,   if true with image num:  35
142 if detected loop: True ,   if true with image num:  35
145 if detected loop: True ,   if true with image num:  35
146 if detected loop: True ,   if true with image num:  35
147 if detected loop: True ,   if true with image num:  35
148 if detected loop: True ,   if true with image num:  35


## 2. Normal SLAM test
this is the tracking part of the slam
1.  take keyframes
2.  detect feature points (SURF here)
3.  match the feature points (Flann matcher)
4.  calculate transformation matrix (for rigid body)
5.  tracking the camera poisition

In [4]:
from IdvSlam import IdvSlam
from pathlib import Path
import os
import numpy as np

In [2]:
doc = 'F:/pfe_2/realsense/frames'
# doc is the dictonary of the input images
# and the second input for the class is '', which is the location to create the output files
slam = IdvSlam(doc, '')

In [16]:
# set the parameter of feature marching
slam.set_match_param(0.7)
# reset the slam (reset the tracking camera position)
slam.reset_Ms()
# calculate from the 50th image and for 150 images
slam.calculate_for_n_images(150, 50)

[========================================================================] 100%


0

In [17]:
# make the points cloud ply file for visualize
slam.make_ply(10)

[========================================================================] 100%


slam result with out loop closing

beginning from the entrance door (blue cycle) and end with the same door (in red cycle), but because of the accumaltion of error, the door at the beginning and at the end are seperated.


![title](img/non_cl_1.PNG)

## 3. Loop closure Slam test
add the loop closure in Test 1 to the normal Slam

In [1]:
from IdvLoopSlam import IdvLoopSlam
import glob
import numpy as np
import matplotlib.pyplot as plt
import winsound
duration = 1000  # millisecond
freq = 440  # Hz

### 3.1 define with training

In [5]:
doc = 'F:/pfe_2/realsense/frames'
slam = IdvLoopSlam(doc, '')
FileList = glob.glob('F:/pfe_2/parking/loop_closure/*.npy')
slam.set_vocabulary_by_train(FileList)
file_dictionary = 'voca.npy'
slam.loopclosure.save_vocabulary(file_dictionary)

--- start training ---
--- train finished ---


### 3.2  define with old trained

In [2]:
doc = 'F:/pfe_2/realsense/frames'
slam = IdvLoopSlam(doc, '')
slam.set_vocabulary('voca.npy')

### 3.3  main process

In [5]:
# set the parameter of feature matching
slam.set_match_param(0.7)
slam.reset_Ms()
# set the maximum image of refine when detecting loop
slam.set_max_image(100)
# set the range for not take in to count when detect loop closure
slam.set_loop_detect_range(20)
# calculate from the 50th image and for 150 images
slam.calculate_for_n_images(150, 50)
# draw all the camera position
slam.draw_camera()

128 image correction of matrix with image 5: ◢                                 
129 image correction of matrix with image 5: ◢                                 
133 image correction of matrix with image 10: ◢                                
136 image correction of matrix with image 13: ◢                                
137 image correction of matrix with image 13: ◢                                
138 image correction of matrix with image 14: ◢                                
139 image correction of matrix with image 15: ◢                                
140 image correction of matrix with image 17: ◢                                
141 image correction of matrix with image 18: ◢                                
142 image correction of matrix with image 17: ◢                                
143 image correction of matrix with image 17: ◢                                
144 image correction of matrix with image 19: ◢                                
145 image correction of matrix with imag

 --- camera drawn


0

In [4]:
slam.make_ply(10) 
#winsound.Beep(freq, duration)

[========================================================================] 100%


the result with loop closure
![title](img/pts_cl.PNG)

the loop is closed compared with the upper result.
as the "two" doors become one single door (same for the sofas)
![title](img/with_cl.PNG)
![title](img/lp0.PNG)

## 3*. Loop closure Slam new test plus
take a new map from the entrance to our office. (upper test only had the map of entrance)

![title](img/lpslam_test2_v2.png)

### 3*.1 test

In [1]:
from IdvLoopSlam import IdvLoopSlam
import glob
import numpy as np
import matplotlib.pyplot as plt
import winsound
duration = 1000  # millisecond
freq = 440  # Hz
doc = 'F:/pfe_2/realsense/frames'
slam = IdvLoopSlam(doc, '')
slam.set_vocabulary('voca.npy')

In [4]:
# set the parameter of feature matching
slam.set_match_param(0.75)
slam.set_ransacThreshold(5)
slam.set_solve_method(0)
slam.reset_Ms()
# set the maximum image of refine when detecting loop
slam.set_max_image(700)
# set the range for not take in to count when detect loop closure
slam.set_loop_detect_range(500)
slam.set_loop_detect_range_max(700)
slam.set_loop_detect_seuil(0.08)
# calculate from the 50th image and for 150 images
slam.calculate_for_n_images(400, 0)
# draw all the camera position
slam.draw_camera()

 --- camera drawn


0

In [5]:
slam.make_ply(40)

[========================================================================] 100%


### 3*.2 error analysis 2018/08/01

2018/08/01 some result may be very bad, possible causes:
1.  loop closure not enough
2.  matrix error
3.  for some images, lack of enough feature points

2018/08/01 possible solutions:
1. --->  make a tour once get into a new room(new region)
2. --->  cannot be very big, and in-evidiable, may by can add a ransac when calculating
3. --->  try to take far away view, or try other feature points
4. --->  reject outlier with calculating transformation matrix

2018/08/02 test result:
1. these method can refine the result
2. error still remains

### 3*.3 arror analysis 2018/08/02

By analysis the sequence of the images, testing over a lots of data. I tested over different scale, different point of view and different density of images.
I found the errors are mainly from the depth error from camera.

For an example, I find the errors came from this series of image:
![title](img/error_ana1.png)
And it is mainly because of this image:
(errors shown in red and green cycles)
![title](img/error_ana2.png)

For most of the cases, the intel realsense camera worked very well. I think this error may came from the fast movement of the camera, or by some other factor.

####  possible solution:
1.  make sure the camera is stable when taking new data.
2.  find a way to detect this error automaticly. Can be the degree if bur of the image, can be corrected by the image neighbors.

### 3*.3 reject the outlier images

![title](img/outlier1.PNG)

## 4.  Mapping test (with segmentation result)

In [1]:
from Mapping import Mapping
import numpy as np
import pickle

In [2]:
doc = 'F:/pfe_2/realsense/frames'
Ms = np.load('IdvSlam/matrix.npy')
mapping = Mapping(doc, '')

In [3]:
mapping.reset()
# imgid is the ids for the image with segmentation outputs
imgid = np.array([50,62,69,79,109,119,149,159,169,179,189,198])
for i in range(len(imgid)):
    idx = imgid[i]-50
    mapping.add_keyframe(idx , Ms[idx])

In [4]:
mapping.draw_Map()

version 1.0 

the points of the objects and the bounding box of these objects
![title](img/obj_bb.PNG)

## 5. Test with updated continue slam

new version of slam

which take one image one time as input,
and can create multi loops and detect loop closure to connect these loops (as in real using, the some image without much features may interrupt the whole process).
![title](img/slam.jpg)

In [ ]:
from IdvLoopSlam import IdvLoopSlam
import glob
import numpy as np
import matplotlib.pyplot as plt
import winsound
duration = 1000  # millisecond
freq = 440  # Hz

In [ ]:
doc = 'F:/pfe_2/realsense/frames'
slam = IdvLoopSlam(doc, '')
slam.set_vocabulary('voca.npy')

In [ ]:
# set the parameter of feature matching
slam.set_match_param(0.7)
slam.reset_Ms()
# set the range for not take in to count when detect loop closure
slam.set_loop_detect_range(20)
for True:
    image = 
    depth = 
    slam.get_one_keyframe(image, depth)
    if :
        break
# draw all the camera position
slam.draw_camera()
slam.save_for_mapping()

#### cnn
[0] Tateno, Keisuke, et al. "CNN-SLAM: Real-time dense monocular SLAM with learned depth prediction." Proceedings of the IEEE Conference on Computer Vision and Pattern Recognition (CVPR). Vol. 2. 2017.

[1] Kendall, Alex, Matthew Grimes, and Roberto Cipolla. "Posenet: A convolutional network for real-time 6-dof camera relocalization." Proceedings of the IEEE international conference on computer vision. 2015.

#### loop closure
[0] Angeli, Adrien, et al. "Fast and incremental method for loop-closure detection using bags of visual words." IEEE Transactions on Robotics 24.5 (2008): 1027-1037.

[1] Latif, Yasir, César Cadena, and José Neira. "Robust loop closing over time for pose graph SLAM." The International Journal of Robotics Research 32.14 (2013): 1611-1626.

[2] Labbe, Mathieu, and Francois Michaud. "Appearance-based loop closure detection for online large-scale and long-term operation." IEEE Transactions on Robotics 29.3 (2013): 734-745.

[3] Weinmann, Martin, Boris Jutzi, and Clément Mallet. "Semantic 3D scene interpretation: a framework combining optimal neighborhood size selection with relevant features." ISPRS Annals of the Photogrammetry, Remote Sensing and Spatial Information Sciences 2.3 (2014): 181.

[4] Csurka, Gabriella, et al. "Visual categorization with bags of keypoints." Workshop on statistical learning in computer vision, ECCV. Vol. 1. No. 1-22. 2004.

#### slam
[1] Pankaj, Dhanya S., and Rama Rao Nidamanuri. "A robust estimation technique for 3D point cloud registration." Image Analysis & Stereology 35.1 (2016): 15-28.

[2] Labbé, Mathieu, and François Michaud. "Memory management for real-time appearance-based loop closure detection." Intelligent Robots and Systems (IROS), 2011 IEEE/RSJ International Conference on. IEEE, 2011.

[3] Zamir, Amir R., et al. "Generic 3d representation via pose estimation and matching." European Conference on Computer Vision. Springer, Cham, 2016.

[4] Engel, Jakob, Thomas Schöps, and Daniel Cremers. "LSD-SLAM: Large-scale direct monocular SLAM." European Conference on Computer Vision. Springer, Cham, 2014.

[5] Shotton, Jamie, et al. "Scene coordinate regression forests for camera relocalization in RGB-D images." Proceedings of the IEEE Conference on Computer Vision and Pattern Recognition. 2013.

#### segmentation
[1] Papadakis, Panagiotis, and David Filliat. "Generic Object Discrimination for Mobile Assistive Robots using Projective Light Diffusion." Computer Vision Workshops (WACVW), 2018 IEEE Winter Applications of. IEEE, 2018.

[2] Landrieu, Loic, and Martin Simonovsky. "Large-scale point cloud semantic segmentation with superpoint graphs." arXiv preprint arXiv:1711.09869 (2017).

[3] Jiang, Hao, and Jianxiong Xiao. "A linear approach to matching cuboids in RGBD images." Proceedings of the IEEE Conference on Computer Vision and Pattern Recognition. 2013.

[4] Song, Shuran, et al. "Semantic scene completion from a single depth image." Computer Vision and Pattern Recognition (CVPR), 2017 IEEE Conference on. IEEE, 2017.